<a href="https://colab.research.google.com/github/QianyueWang0212/mgmt467-analytics-portfolio/blob/main/Labs/Lab4_Regression_BQML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [ ]:
from google.colab import files
import os

# Prompt the user to upload their kaggle.json file.
# This file contains your Kaggle API credentials.
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

# Create the .kaggle directory if it doesn't exist.
# This is where Kaggle expects to find the credentials file.
os.makedirs('/root/.kaggle', exist_ok=True)

# Save the uploaded file to the correct location.
# Using the first uploaded file as we expect only one (kaggle.json).
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])

# Set file permissions to 0600 (owner read/write only).
# This is crucial for security to prevent other users from accessing your API key.
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Verify the Kaggle installation by printing the version.
# This confirms the CLI is installed and can access the credentials.
!kaggle --version

# Done: Kaggle setup

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle (3).json to kaggle (3).json
Kaggle API 1.7.4.5


### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [ ]:
#EXAMPLE (from LLM) — Auth + Project/Region (commented; write your own cell using the prompt)
from google.colab import auth
auth.authenticate_user()

import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # keep consistent; change if instructed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
print("Project:", PROJECT_ID, "| Region:", REGION)

# Set active project for gcloud/BigQuery CLI
!gcloud config set project $GOOGLE_CLOUD_PROJECT
!gcloud config get-value project
# Done: Auth + Project/Region set

Enter your GCP Project ID: mgmt-46700
Project: mgmt-46700 | Region: us-central1
Updated property [core/project].
mgmt-46700


### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


In [ ]:
# Verification step: Check active project and region
!gcloud config get-value project
import os
print("Project:", PROJECT_ID, "| Region:", REGION)

mgmt-46700
Project: mgmt-46700 | Region: us-central1


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [ ]:
# EXAMPLE (from LLM) — Kaggle setup (commented)
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

import os
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only

!kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle (3).json to kaggle (3) (1).json
Kaggle API 1.7.4.5


## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `mahoora00135/flights`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [ ]:
# Create the directory to store raw data
!mkdir -p /content/data/raw

# Download the dataset using Kaggle CLI to /content/data
# The -d flag specifies the dataset, and -p specifies the download path
!kaggle datasets download -d mahoora00135/flights -p /content/data

# Unzip the downloaded dataset into the raw data directory
# -o flag overwrites files if they exist
!unzip -o /content/data/*.zip -d /content/data/raw

# List all CSV files in the raw data directory with their sizes in a neat table
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/mahoora00135/flights
License(s): CC0-1.0
  0% 0.00/10.3M [00:00<?, ?B/s]
100% 10.3M/10.3M [00:00<00:00, 1.80GB/s]
Archive:  /content/data/flights.zip
  inflating: /content/data/raw/flights.csv  
-rw-r--r-- 1 root root 41M Sep 26  2023 /content/data/raw/flights.csv


In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/content/data/raw/flights.csv')

# Display the first few rows of the DataFrame
display(df.head())

,id,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,...,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,name
0,0,2013,1,1,517.0,515,2.0,830.0,819,11.0,...,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,United Air Lines Inc.
1,1,2013,1,1,533.0,529,4.0,850.0,830,20.0,...,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,United Air Lines Inc.
2,2,2013,1,1,542.0,540,2.0,923.0,850,33.0,...,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,American Airlines Inc.
3,3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,...,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,JetBlue Airways
4,4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,...,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,Delta Air Lines Inc.


Feature Brainstorming (Gemini Prompt)
Use Gemini to brainstorm predictive features:

prompt = """
```
# TASK: Brainstorm features for a machine learning model.
# CONTEXT: I'm using the BigQuery public flights.csv dataset. I want to predict the 'arr_delay' (arrival delay in minutes), which is a numerical value.
# GOAL: List 5 columns from the dataset that you think would be the best predictors for 'arr_delay' and briefly explain why for each one.
"""
```
Add your chosen features below.

| **Feature**              | **Why It’s Useful for Predicting Arrival Delay**                                                                                          |
| ------------------------ | ----------------------------------------------------------------------------------------------------------------------------------------- |
| **`dep_delay`**          | Departure delay is the strongest direct predictor — flights that leave late tend to arrive late, often with only partial recovery in-air. |
| **`distance`**           | Longer routes may allow more time to make up or worsen delays, while shorter routes offer less flexibility.                               |
| **`air_time`**           | Indicates flight duration — combined with `distance`, it can reveal route efficiency and potential buffer time.                           |
| **`hour`**               | Time of day affects congestion; early-morning flights are often on time, while afternoon/evening flights may experience cascading delays. |
| **`origin` (or `dest`)** | Some airports are chronically congested or weather-affected, introducing systematic delay patterns.                                       |


In [ ]:
# Create a GCS bucket (only once per project)
BUCKET_NAME = f"{PROJECT_ID}-flights-bucket"
!gsutil mb -l {REGION} gs://{BUCKET_NAME}/ || echo "Bucket may already exist"

# Upload dataset to the bucket
!gsutil cp /content/data/raw/flights.csv gs://{BUCKET_NAME}/flights.csv

print(f"✅ Uploaded flights.csv to: gs://{BUCKET_NAME}/flights.csv")


Creating gs://mgmt-46700-flights-bucket/...
Copying file:///content/data/raw/flights.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/40.9 MiB.                                     
✅ Uploaded flights.csv to: gs://mgmt-46700-flights-bucket/flights.csv


In [ ]:
from google.cloud import bigquery
from google.api_core.exceptions import Conflict

client = bigquery.Client(project=PROJECT_ID)

dataset_id = f"{PROJECT_ID}.flights_dataset"
table_id = f"{dataset_id}.flights"

# Create dataset if it doesn't exist
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION

try:
    client.create_dataset(dataset, timeout=30)  # Make an API request.
    print(f"✅ Dataset {dataset_id} created.")
except Conflict:
    print(f"Dataset {dataset_id} already exists.")
except Exception as e:
    print(f"Error creating dataset {dataset_id}: {e}")
    # Exit the cell execution if dataset creation fails
    raise

# Define load job from GCS
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

uri = f"gs://{BUCKET_NAME}/flights.csv"
load_job = client.load_table_from_uri(uri, table_id, job_config=job_config)
load_job.result()

table = client.get_table(table_id)
print(f"✅ Loaded {table.num_rows} rows into {table_id}")

✅ Dataset mgmt-46700.flights_dataset created.
✅ Loaded 336776 rows into mgmt-46700.flights_dataset.flights



---
## Train Your First Regression Model

Use **BQML** to create a **linear regression** model. Edit the feature list as needed.  
We also enable explanations to support **ML.EXPLAIN_PREDICT** and global feature importance.


In [ ]:
query = f"""
CREATE OR REPLACE MODEL `{dataset_id}.flight_delay_predictor`
OPTIONS(
  model_type='LINEAR_REG',
  input_label_cols=['arr_delay']
) AS
SELECT
  arr_delay,
  dep_delay,
  distance,
  air_time,
  hour,
  origin
FROM `{table_id}`
WHERE arr_delay IS NOT NULL
LIMIT 200000;
"""
client.query(query).result()
print("✅ Model created successfully: flight_delay_predictor")

✅ Model created successfully: flight_delay_predictor



---
## Evaluate Model Performance (`ML.EVALUATE`) — Validate

Run evaluation to view **mean_absolute_error**, **r2_score**, etc. Then use Gemini to explain a key metric to a non-technical audience.

**Gemini Explainer Prompt:**

```python
prompt = """
# TASK: Explain a model evaluation metric in simple language.
# CONTEXT: I just ran ML.EVALUATE on my BigQuery ML linear regression model predicting flight arrival delays (arr_delay).
# The key metric I want to explain is 'r2_score' (coefficient of determination).
# GOAL: Explain what this metric means to a non-technical stakeholder in one short paragraph,
# focusing on practical business understanding, not math.
"""

```


In [ ]:
# Evaluate model performance
eval_query = f"""
SELECT *
FROM ML.EVALUATE(MODEL `{dataset_id}.flight_delay_predictor`);
"""

eval_df = client.query(eval_query).to_dataframe()
display(eval_df)


,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,10.488247,230.576599,0.937158,7.697563,0.884705,0.88473


Based on the prompt and the model evaluation results (where the r2_score is 0.884705), here's a simple explanation of what the R2 score means for your flight delay model:

The R-squared score tells us how well our model's predictions match the actual flight delays. An R-squared of 0.88 (or 88%) means that about 88% of the variation in flight arrival delays can be explained by the features we included in the model (like departure delay, distance, etc.). A higher R-squared indicates a better fit, meaning the model is doing a good job of capturing the patterns in the data that lead to delays.


---
## Explain Predictions (`ML.EXPLAIN_PREDICT`) — Why did the model predict that?

Author your own prompt to generate a query that explains a **hypothetical** case:  
> Predict the arrival delay for a 2000-mile flight with a 30-minute departure delay on carrier 'AA', and explain the top contributing features.

**Hint:** You’ll need a small input table with the required features.

> **Your Gemini prompt (write in the next cell):**  
> *Ask for a full `ML.EXPLAIN_PREDICT` example for this model and scenario, returning top features.*


prompt =
```
# TASK: Generate a full ML.EXPLAIN_PREDICT example query in BigQuery ML.
# CONTEXT: I have a linear regression model called `flight_delay_predictor` in BigQuery ML.
# GOAL: Predict and explain why the model made its prediction for a hypothetical flight.
# SCENARIO:
# - dep_delay = 30 minutes
# - distance = 2000 miles
# - air_time = 250 minutes
# - hour = 10
# - origin = 'JFK'
# - carrier = 'AA'
# REQUIREMENTS:
# - Use ML.EXPLAIN_PREDICT to show the predicted arrival delay ('arr_delay')
# - Return the top contributing features and their weights or SHAP values.
# - Include comments describing each step.
```

In [ ]:
explain_query = f"""
SELECT *
FROM ML.EXPLAIN_PREDICT(MODEL `{dataset_id}.flight_delay_predictor`,
  (SELECT
      30 AS dep_delay,
      2000 AS distance,
      250 AS air_time,
      10 AS hour,
      'JFK' AS origin,
      'AA' AS carrier),
  STRUCT(3 AS top_k_features)
);
"""

explain_pred_df = client.query(explain_query).to_dataframe()
display(explain_pred_df)


,predicted_arr_delay,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,dep_delay,distance,air_time,hour,origin,carrier
0,3.507384,"[{'feature': 'origin', 'attribution': -6708.25...",6716.795383,3.507384,0.0,30,2000,250,10,JFK,AA
